In [2]:
import requests

from scrapy import Selector
import plotly.express as px

import pandas as pd
pd.options.mode.chained_assignment = None
pd.set_option('display.float_format', lambda x: '%.3f' % x)

import base

with open('/Users/ilya/Desktop/keys/yahoo_finance.txt','r') as file:
    key = file.readlines()

KEY = key[0]

from sqlalchemy import create_engine

In [11]:
url = 'https://en.wikipedia.org/wiki/Nasdaq-100'

html = requests.get(url).text
sel = Selector(text = html)

nasdaq100_df = pd.DataFrame()

for n in range(2,104):
    ticker = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[2]'.format(n)).extract()[0].strip('</td>')
    company_name = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[1]/a/@title'.format(n)).get()
    industry = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[3]'.format(n)).get().strip('</td>')
    sub_industry = sel.xpath('//table[@id = "constituents"]/tbody/tr[{}]/td[3]'.format(n)).get().strip('</td>')
    new_list = [ticker, company_name, industry, sub_industry]
    pd_series_row = pd.Series(new_list, index = ['ticker', 'company_name', 'industry', 'sub_industry'])
    nasdaq100_df = nasdaq100_df.append(pd_series_row, ignore_index=True)
    print(ticker, end = ' ')

nasdaq100_df.head(4)

ATVI ADBE AMD ALGN GOOGL GOOG AMZN AEP AMGN ADI ANSS AAPL AMAT ASML TEAM ADSK ADP BIDU BIIB BKNG AVGO CDNS CDW CERN CHTR CHKP CTAS CSCO CTSH CMCSA CPRT COST CRWD CSX DXCM DOCU DLTR EBAY EA EXC FAST FISV FOXA FOX GILD HON IDXX ILMN INCY INTC INTU ISRG JD KDP KLAC KHC LRCX LULU MAR MRVL MTCH MELI FB MCHP MU MSFT MRNA MDLZ MNST NTES NFLX NVDA NXPI ORLY OKTA PCAR PAYX PYPL PTON PEP PDD QCOM REGN ROST SGEN SIRI SWKS SPLK SBUX SNPS TMUS TSLA TXN TCOM VRSN VRSK VRTX WBA WDAY XEL XLNX ZM 

,company_name,industry,sub_industry,ticker
0,Activision Blizzard,Communication Services,Communication Services,ATVI
1,Adobe Inc.,Information Technology,Information Technology,ADBE
2,Advanced Micro Devices,Information Technology,Information Technology,AMD
3,Align Technology,Health Care,Health Care,ALGN


In [12]:
counter = 0

stock_history_df = pd.DataFrame(columns = ['ticker','yoy_growth','std','mean_price','volatility','last_close_price'])
la_ratio_main_df = pd.DataFrame()
income_statement_main_df = pd.DataFrame()
pe_ratio_main_df = pd.DataFrame()

#print('Geting year stock prices...')
for ticker_name in list(nasdaq100_df['ticker'][0:3]):
    series_row = base.list_perfomance(ticker_name)
    stock_history_df = stock_history_df.append(series_row, ignore_index=True)

    mini_df = base.get_la_ratio(ticker_name,1)
    la_ratio_main_df = la_ratio_main_df.append(mini_df)

    mini_df = base.get_income_statement(ticker_name,1)
    income_statement_main_df = income_statement_main_df.append(mini_df)

    mini_df = base.get_pe_ratio(ticker_name,1)
    pe_ratio_main_df = pe_ratio_main_df.append(mini_df)
    
    print(ticker_name, end = ' ')

print('Done!')

ATVI ADBE AMD Done!


In [13]:
final_df = stock_history_df.merge(la_ratio_main_df, left_on = ['ticker'], right_on = ['ticker'])\
                        .merge(income_statement_main_df, left_on = ['ticker'], right_on = ['ticker'])\
                        .merge(pe_ratio_main_df, left_on = ['ticker'], right_on = ['ticker'])

final_df = final_df.merge(nasdaq100_df, left_on = ['ticker'], right_on = ['ticker'])

final_df = final_df[['ticker','company_name','industry','sub_industry','last_close_price','yoy_growth','volatility',
                    'assets_billion','liabilities_billion','la_ratio','gross_profit_margin','net_profit_margin',
                    'pe_ratio']]

final_df

,ticker,company_name,industry,sub_industry,last_close_price,yoy_growth,volatility,assets_billion,liabilities_billion,la_ratio,gross_profit_margin,net_profit_margin,pe_ratio
0,ATVI,Activision Blizzard,Communication Services,Communication Services,58.600,-0.363,0.110,11.210,2.020,0.180,0.745,0.309,23.24
1,ADBE,Adobe Inc.,Information Technology,Information Technology,669.800,0.285,0.140,8.620,6.190,0.720,0.881,0.308,57.56
2,AMD,Advanced Micro Devices,Information Technology,Information Technology,158.400,0.415,0.200,7.990,3.560,0.450,0.484,0.214,36.49


In [14]:
#final_df.to_csv('/Users/ilya/Desktop/Computer-Science/Github/check-stock-portfolio/saved_df.csv')
final_df = pd.read_csv('/Users/ilya/Desktop/Computer-Science/Github/check-stock-portfolio/saved_df.csv').drop(columns = ['Unnamed: 0'])

In [15]:
#considerations
#0. yoy_growth
#1. volatility
#2. la_ratio
#3. gross_profit_margin
#4. net_profit_margin
#5. pe_ratio

In [26]:
analysis_df = final_df[['ticker','yoy_growth','volatility','la_ratio','gross_profit_margin','net_profit_margin','pe_ratio']]

analysis_df.describe()

,yoy_growth,volatility,la_ratio,gross_profit_margin,net_profit_margin,pe_ratio
count,101.000,102.000,102.000,102.000,102.000,94.000
mean,0.106,0.108,0.713,0.571,0.146,47.786
std,0.305,0.062,0.539,0.207,0.242,49.756
min,-1.349,0.040,0.140,-0.278,-0.877,7.770
25%,-0.004,0.060,0.340,0.463,0.106,21.945
50%,0.157,0.090,0.595,0.576,0.175,34.715
75%,0.262,0.130,0.865,0.729,0.279,56.880
max,0.615,0.440,3.500,0.917,0.671,401.800


In [41]:
yoy_growth_bottom = analysis_df['yoy_growth'].mean() - analysis_df['yoy_growth'].std()

la_analysis_df = analysis_df[analysis_df['yoy_growth'] > yoy_growth_bottom]
la_analysis_df.loc[:,'yoy_growth_bin'] = pd.cut(la_analysis_df['yoy_growth'], 8, precision = 1)
la_analysis_df2 = la_analysis_df[['ticker','yoy_growth_bin']].groupby('yoy_growth_bin').count().reset_index()
la_analysis_df2.loc[:,'yoy_growth_bin'] = la_analysis_df2['yoy_growth_bin'].astype('str')


fig = px.bar(la_analysis_df2, x='yoy_growth_bin', y='ticker', text = 'ticker')

fig.update_traces(textposition='outside',
                  texttemplate='%{text}',
                  marker_color='#917150',
                  textfont_size=14)

fig.update_layout(title_text = 'YoY Stock Price Growth',
                  yaxis_title = 'Number of Companies',
                  xaxis_title = 'Growth Rate Bin',
                  height = 500, width = 1000)

fig.add_annotation(x=7, y=20,
            text="median: {}".format(analysis_df['yoy_growth'].median()),
            showarrow=False)

fig.update_layout(showlegend=False)

fig.show()

In [43]:
la_ratio_top = analysis_df['la_ratio'].mean() + analysis_df['la_ratio'].std()

la_analysis_df = analysis_df[analysis_df['la_ratio'] < la_ratio_top]
la_analysis_df.loc[:,'la_ratio_bin'] = pd.cut(la_analysis_df['la_ratio'], 8, precision = 1)
la_analysis_df2 = la_analysis_df[['ticker','la_ratio_bin']].groupby('la_ratio_bin').count().reset_index()
la_analysis_df2.loc[:,'la_ratio_bin'] = la_analysis_df2['la_ratio_bin'].astype('str')


fig = px.bar(la_analysis_df2, x='la_ratio_bin', y='ticker', text = 'ticker')

fig.update_traces(textposition='outside',
                  texttemplate='%{text}',
                  marker_color='#106e45',
                  textfont_size=14)

fig.update_layout(title_text = 'Liabilities to Assets Distribution',
                  yaxis_title = 'Number of Companies',
                  xaxis_title = 'LA Ratio Bin',
                  height = 500, width = 1000)

fig.add_annotation(x=7, y=16,
            text="median: {}".format(analysis_df['la_ratio'].median()),
            showarrow=False)

fig.update_layout(showlegend=False)

fig.show()

In [44]:
volatility_top = analysis_df['volatility'].mean() + analysis_df['volatility'].std()

volatility_analysis_df = analysis_df[analysis_df['volatility'] < volatility_top]
volatility_analysis_df.loc[:,'volatility_bin'] = pd.cut(volatility_analysis_df['volatility'], 8, precision = 1)
volatility_analysis_df2 = volatility_analysis_df[['ticker','volatility_bin']].groupby('volatility_bin').count().reset_index()
volatility_analysis_df2.loc[:,'volatility_bin'] = volatility_analysis_df2['volatility_bin'].astype('str')

fig = px.bar(volatility_analysis_df2, x='volatility_bin', y='ticker', text = 'ticker')

fig.update_traces(textposition='outside',
                  texttemplate='%{text}',
                  marker_color='#9467bd',
                  textfont_size=14)

fig.update_layout(title_text = 'Volatility Ratio Distribution',
                  yaxis_title = 'Number of Companies',
                  xaxis_title = 'Volatility Bin',
                  height = 500, width = 1000)

fig.add_annotation(x=7, y=16,
            text="median: {}".format(analysis_df['volatility'].median()),
            showarrow=False)

fig.update_layout(showlegend=False)

fig.show()

In [52]:
gpm_analysis_df = analysis_df[analysis_df['gross_profit_margin'] >= 0][['ticker','gross_profit_margin']]
gpm_analysis_df.loc[:,'gross_profit_margin_bin'] = pd.cut(gpm_analysis_df['gross_profit_margin'], 8, precision = 1)
gpm_analysis_df2 = gpm_analysis_df[['gross_profit_margin_bin','ticker']].groupby('gross_profit_margin_bin').count().reset_index()
gpm_analysis_df2.loc[:,'gross_profit_margin_bin'] = gpm_analysis_df2['gross_profit_margin_bin'].astype('str')

fig = px.bar(gpm_analysis_df2, x='gross_profit_margin_bin', y='ticker', text = 'ticker')

fig.update_traces(textposition='outside',
                  texttemplate='%{text}',
                  marker_color='#000080',
                  textfont_size=14)

fig.update_layout(title_text = 'Gross Profit Margin Distribution',
                  yaxis_title = 'Number of Companies',
                  xaxis_title = 'Gross Profit Margin Bin',
                  height = 500, width = 1000)

fig.add_annotation(x=7, y=22,
            text="median: {gpm:.3f}".format(gpm = analysis_df['gross_profit_margin'].median()),
            showarrow=False)

fig.update_layout(showlegend=False)

#fig.update_layout({'plot_bgcolor': 'rgba(192, 218, 218, 0)','paper_bgcolor': 'rgba(192, 218, 218, 0)',})

fig.show()

In [54]:
gpm_analysis_df = analysis_df[analysis_df['net_profit_margin'] >= 0][['ticker','net_profit_margin']]
gpm_analysis_df.loc[:,'net_profit_margin_bin'] = pd.cut(gpm_analysis_df['net_profit_margin'], 8, precision = 1)
gpm_analysis_df2 = gpm_analysis_df[['net_profit_margin_bin','ticker']].groupby('net_profit_margin_bin').count().reset_index()
gpm_analysis_df2.loc[:,'net_profit_margin_bin'] = gpm_analysis_df2['net_profit_margin_bin'].astype('str')

fig = px.bar(gpm_analysis_df2, x='net_profit_margin_bin', y='ticker', text = 'ticker')

fig.update_traces(textposition='outside',
                  texttemplate='%{text}',
                  marker_color='#5AA295',
                  textfont_size=14)

fig.update_layout(title_text = 'Net Profit Margin Distribution',
                  yaxis_title = 'Number of Companies',
                  xaxis_title = 'Net Profit Margin Bin',
                  height = 500, width = 1000)

fig.add_annotation(x=7, y=30,
            text="median: {npm:.3f}".format(npm = analysis_df['net_profit_margin'].median()),
            showarrow=False)

fig.update_layout(showlegend=False)

fig.show()

In [55]:
pe_ratio_top = analysis_df['pe_ratio'].mean() + analysis_df['pe_ratio'].std()

pe_analysis_df = analysis_df[analysis_df['pe_ratio'] < pe_ratio_top]
pe_analysis_df.loc[:,'pe_ratio_bin'] = pd.cut(pe_analysis_df['pe_ratio'], 8, precision = 1)
pe_analysis_df2 = pe_analysis_df[['ticker','pe_ratio_bin']].groupby('pe_ratio_bin').count().reset_index()
pe_analysis_df2.loc[:,'pe_ratio_bin'] = pe_analysis_df2['pe_ratio_bin'].astype('str')


fig = px.bar(pe_analysis_df2, x='pe_ratio_bin', y='ticker', text = 'ticker')

fig.update_traces(textposition='outside',
                  texttemplate='%{text}',
                  marker_color='#917150',
                  textfont_size=14)

fig.update_layout(title_text = 'PE Ratio',
                  yaxis_title = 'Number of Companies',
                  xaxis_title = 'PE Ratio Bin',
                  height = 500, width = 1000)

fig.add_annotation(x=7, y=26,
            text="median: {pe:.3f}".format(pe = analysis_df['pe_ratio'].median()),
            showarrow=False)

fig.update_layout(showlegend=False)

fig.show()

In [3]:
analysis_df = pd.read_csv('/Users/ilya/Desktop/Computer-Science/Github/choosy-stock/saved_df.csv').drop(columns = ['Unnamed: 0'])

In [4]:
analysis_df

,ticker,company_name,industry,sub_industry,last_close_price,yoy_growth,volatility,assets_billion,liabilities_billion,la_ratio,gross_profit_margin,net_profit_margin,pe_ratio
0,ATVI,Activision Blizzard,Communication Services,Communication Services,60.600,-0.289,0.110,11.210,2.020,0.180,0.745,0.309,23.240
1,ADBE,Adobe Inc.,Information Technology,Information Technology,662.100,0.280,0.140,8.620,6.190,0.720,0.881,0.308,57.560
2,AMD,Advanced Micro Devices,Information Technology,Information Technology,154.800,0.437,0.190,7.990,3.560,0.450,0.484,0.214,36.490
3,ALGN,Align Technology,Health Care,Health Care,634.200,0.250,0.110,2.460,1.790,0.730,0.743,0.178,76.220
4,GOOGL,Alphabet Inc.,Communication Services,Communication Services,2843.700,0.372,0.170,184.110,61.780,0.340,0.576,0.291,29.010
...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,WBA,Walgreens Boots Alliance,Consumer Staples,Consumer Staples,45.900,0.150,0.080,15.810,22.050,1.390,0.219,0.018,19.590
98,WDAY,"Workday, Inc.",Information Technology,Information Technology,277.500,0.193,0.080,4.690,4.280,0.910,0.729,0.032,NaN
99,XEL,Xcel Energy,Utilities,Utilities,64.800,-0.043,0.050,4.490,5.760,1.280,0.453,0.176,21.260
100,XLNX,Xilinx,Information Technology,Information Technology,223.900,0.386,0.160,4.190,0.610,0.150,0.686,0.251,49.180


In [8]:
analysis_df2 = analysis_df.dropna()

In [ ]:
analysis_df2

In [9]:
analysis_df2[(analysis_df2['last_close_price'] <= 500) &
         (analysis_df2['yoy_growth'] >= 0.1) &
         (analysis_df2['la_ratio'] <= 0.7) &
         (analysis_df2['gross_profit_margin'] >= 0.4) &
         (analysis_df2['net_profit_margin'] >= 0.1) &
         (analysis_df2['pe_ratio'] <= 50)
         ].sort_values(by = 'pe_ratio', ascending = True)

,ticker,company_name,industry,sub_industry,last_close_price,yoy_growth,volatility,assets_billion,liabilities_billion,la_ratio,gross_profit_margin,net_profit_margin,pe_ratio
43,FOX,Fox Corporation,Communication Services,Communication Services,35.100,0.182,0.100,8.880,2.870,0.320,0.484,0.230,10.280
42,FOXA,Fox Corporation,Communication Services,Communication Services,37.600,0.223,0.110,8.880,2.870,0.320,0.484,0.230,11.110
81,QCOM,Qualcomm,Information Technology,Information Technology,175.700,0.182,0.090,20.080,11.950,0.600,0.578,0.300,16.120
86,SWKS,Skyworks Solutions,Information Technology,Information Technology,152.500,0.107,0.080,2.870,0.660,0.230,0.468,0.249,19.410
64,MU,Micron Technology,Information Technology,Information Technology,83.400,0.230,0.090,19.910,6.420,0.320,0.479,0.329,20.360
37,EBAY,EBay,Consumer Discretionary,Consumer Discretionary,72.500,0.292,0.130,17.760,5.300,0.300,0.729,0.106,21.240
27,CSCO,Cisco Systems,Information Technology,Information Technology,54.700,0.219,0.090,37.590,23.240,0.620,0.624,0.231,22.390
12,AMAT,Applied Materials,Information Technology,Information Technology,143.300,0.423,0.150,16.110,6.340,0.390,0.481,0.280,23.770
54,KLAC,KLA Corporation,Information Technology,Information Technology,399.000,0.363,0.120,6.140,2.520,0.410,0.609,0.512,25.020
62,FB,Meta Platforms,Communication Services,Communication Services,333.100,0.166,0.120,75.420,17.810,0.240,0.801,0.317,25.180


In [25]:
analysis_df3 = analysis_df2[['ticker','company_name','industry','last_close_price','yoy_growth','volatility','la_ratio','net_profit_margin','pe_ratio']]

In [5]:
cnx = create_engine()

TypeError: create() missing 1 required positional argument: 'name_or_url'

In [19]:
analysis_df3 = analysis_df3.reset_index().rename(columns = {'index':'id'})

In [23]:
analysis_df3.to_sql('company', cnx, schema = 'public', index = False, chunksize=100, if_exists='replace', method = 'multi')

In [1]:
analysis_df3

NameError: name 'analysis_df3' is not defined